In [6]:
# ---------- Imports ----------
from fpdf import FPDF
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq 
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from google_auth_oauthlib.flow import InstalledAppFlow
import os
import base64

# Load environment variables
load_dotenv()

# ---------- Configuration ----------
CLIENT_SECRETS_FILE = 'client_secret.json'  # From Google Cloud Console
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
SENDER_EMAIL = "syedaliahmed171@gmail.com"

# ---------- Invoice Data ----------
invoice_data = {
    'customer_name': 'John Doe',
    'customer_email': 'syedaliahmedshah677@gmail.com',
    'date': '2025-04-29',
    'amount': '2500',
    'due_date': '2025-05-10'
}

# ---------- Gmail Auth Functions ----------
def get_gmail_credentials():
    """Get OAuth 2.0 credentials"""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# ---------- Function: Create PDF ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)

    pdf.output(filename)

# ---------- Function: Send Email via Gmail API ----------
def send_invoice_via_gmail(to_email, subject, body, attachment_path):
    try:
        # Authenticate
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        # Create email
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))
        
        # Attach PDF
        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        # Send
        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(
            userId="me",
            body={"raw": raw_msg}
        ).execute()
        print("✅ Email sent via Gmail API!")
        return True
        
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False

# ---------- LangChain Tools ----------
def tool_create_invoice(_):
    try:
        filename = "invoice.pdf"
        create_invoice_pdf(invoice_data, filename)
        if os.path.exists(filename) and os.path.getsize(filename) > 0:
            return "PDF invoice created successfully."
        return "Error: Failed to create PDF invoice."
    except Exception as e:
        return f"Error creating invoice: {str(e)}"

def tool_send_invoice(_):
    try:
        success = send_invoice_via_gmail(
            to_email=invoice_data['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path="invoice.pdf"
        )
        return "Invoice sent successfully." if success else "Failed to send invoice."
    except Exception as e:
        return f"Error sending invoice: {str(e)}"

tools = [
    Tool(name="CreateInvoice", func=tool_create_invoice, 
         description="Create a PDF invoice. Input is ignored."),
    Tool(name="SendInvoice", func=tool_send_invoice,
         description="Send the invoice via email. Input is ignored.")
]

# ---------- LangChain Agent ----------
try:
    llm = ChatGroq(
        model="llama3-8b-8192",
        temperature=0.0,
        api_key=GROQ_API_KEY
    )

    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3,  # Hard limit
        early_stopping_method="generate"
    )

    # ---------- Run Agent ----------
    print("Starting invoice process...")
    task = "Create and send an invoice to the customer."
    result = agent.run(task)
    print(result)
except Exception as e:
    print(f"Error initializing agent: {str(e)}")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Starting invoice process...
Thought: I need to create an invoice first, and then send it to the customer. I'll need to decide on the details of the invoice, such as the date, customer information, and items to be invoiced.

Action: CreateInvoice
Action Input: None
Observation: PDF invoice created successfully.
Thought:Thought: Now that I have created the invoice, I need to decide on the details of the invoice, such as the date, customer information, and items to be invoiced. I'll need to fill in the necessary information.

Action: CreateInvoice
Action Input: None
Observation: PDF invoice created successfully.
Thought:Action: SendInvoice
Action Input: None✅ Email sent via Gmail API!

Observation: Invoice sent successfully.
Thought:Here is the answer:

Question: Create and send an invoice to the customer.

Thought: I need to create an invoice first, and then send it to the customer. I'll need to decide on the details of the invoice, such as the date, customer information, and items to be

In [10]:
# ---------- Imports ----------
from fpdf import FPDF
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq 
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from google_auth_oauthlib.flow import InstalledAppFlow
import os
import base64

# Load environment variables
load_dotenv()

# ---------- Configuration ----------
CLIENT_SECRETS_FILE = 'client_secret.json'  # From Google Cloud Console
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
SENDER_EMAIL = "syedaliahmed171@gmail.com"

# ---------- Invoice Data ----------
invoice_data = [
    {
        'customer_name': 'John Doe',
        'customer_email': 'syedaliahmedshah677@gmail.com',
        'date': '2025-04-29',
        'amount': '2500',
        'due_date': '2025-03-10',
        'status': 'unpaid'  # unpaid or paid
    },
    {
        'customer_name': 'Alice Smith',
        'customer_email': 'alice@example.com',
        'date': '2025-04-15',
        'amount': '1800',
        'due_date': '2025-04-20',
        'status': 'paid'
    }
]


# ---------- Gmail Auth Functions ----------
def get_gmail_credentials():
    """Get OAuth 2.0 credentials"""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# ---------- Function: Create PDF ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)

    pdf.output(filename)

# ---------- Function: Send Email via Gmail API ----------
def send_invoice_via_gmail(to_email, subject, body, attachment_path):
    try:
        # Authenticate
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        # Create email
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))
        
        # Attach PDF
        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        # Send
        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(
            userId="me",
            body={"raw": raw_msg}
        ).execute()
        print("✅ Email sent via Gmail API!")
        return True
        
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False

# ---------- LangChain Tools ----------
def tool_create_invoice(_):
    try:
        filename = "invoice.pdf"
        create_invoice_pdf(invoice_data, filename)
        if os.path.exists(filename) and os.path.getsize(filename) > 0:
            return "PDF invoice created successfully."
        return "Error: Failed to create PDF invoice."
    except Exception as e:
        return f"Error creating invoice: {str(e)}"

def tool_send_invoice(_):
    try:
        success = send_invoice_via_gmail(
            to_email=invoice_data['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path="invoice.pdf"
        )
        return "Invoice sent successfully." if success else "Failed to send invoice."
    except Exception as e:
        return f"Error sending invoice: {str(e)}"
    
from datetime import datetime

def tool_remind_unpaid_invoices(_):
    today = datetime.today().strftime('%Y-%m-%d')
    reminders_sent = []

    for invoice in invoice_data:
        if invoice['status'] == 'unpaid' and invoice['due_date'] < today:
            filename = f"reminder_{invoice['customer_name'].replace(' ', '_')}.pdf"
            create_invoice_pdf(invoice, filename)
            
            success = send_invoice_via_gmail(
                to_email=invoice['customer_email'],
                subject="Payment Reminder: Unpaid Invoice",
                body="Hello, this is a reminder for your pending invoice. Please make payment as soon as possible.",
                attachment_path=filename
            )
            if success:
                reminders_sent.append(invoice['customer_name'])

    return f"✅ Reminders sent to: {', '.join(reminders_sent)}" if reminders_sent else "✅ No unpaid overdue invoices."

def mark_invoice_as_paid(customer_email):
    updated = False
    for invoice in invoice_data:
        if invoice['customer_email'] == customer_email and invoice['status'] == 'unpaid':
            invoice['status'] = 'paid'
            updated = True
    return f"✅ Invoice marked as paid for {customer_email}" if updated else f"⚠️ No unpaid invoice found for {customer_email}"


tools = [
    Tool(name="CreateInvoice", func=tool_create_invoice, 
         description="Create a PDF invoice. Input is ignored."),

    Tool(name="SendInvoice", func=tool_send_invoice,
         description="Send the invoice via email. Input is ignored."),
    Tool(name="RemindUnpaidInvoices", func=tool_remind_unpaid_invoices,
     description="Scan unpaid invoices and send reminders if overdue."),
    Tool(name="MarkInvoicePaid", func=lambda x: mark_invoice_as_paid(x),
     description="Mark an invoice as paid. Input should be the customer's email address.")
 
]

# ---------- LangChain Agent ----------
try:
    llm = ChatGroq(
        model="llama3-8b-8192",
        temperature=0.0,
        api_key=GROQ_API_KEY
    )

    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3,  # Hard limit
        early_stopping_method="generate"
    )

    # ---------- Run Agent ----------
    print("Starting invoice process...")
    task = "For all customers, if an invoice doesn't exist, create and send it. If the invoice is unpaid and past due, send a reminder. After payment, mark as paid"
    result = agent.run(task)
    print(result)

except Exception as e:
    print(f"Error initializing agent: {str(e)}")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Starting invoice process...
Thought: I need to check if an invoice exists for each customer, and if not, create and send it. Then, I need to check if the invoice is unpaid and past due, and if so, send a reminder. Finally, if the invoice is paid, mark it as paid.

Action: CreateInvoice(_)
Action Input: None
Observation: CreateInvoice(_) is not a valid tool, try one of [CreateInvoice, SendInvoice, RemindUnpaidInvoices, MarkInvoicePaid].
Thought:Action: CreateInvoice(None)
Action Input: None
Observation: CreateInvoice(None) is not a valid tool, try one of [CreateInvoice, SendInvoice, RemindUnpaidInvoices, MarkInvoicePaid].
Thought:Action: CreateInvoice(_)
Action Input: None
Observation: CreateInvoice(_) is not a valid tool, try one of [CreateInvoice, SendInvoice, RemindUnpaidInvoices, MarkInvoicePaid].
Thought:Final Answer: This task is not possible with the given tools. The CreateInvoice(_) function is not a valid tool, and it cannot be used to create an invoice.

> Finished chain.
This

In [15]:
# ---------- Imports ----------
from fpdf import FPDF
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from google_auth_oauthlib.flow import InstalledAppFlow
from datetime import datetime
import os
import base64
from dotenv import load_dotenv

# ---------- Load Environment ----------
load_dotenv()

# ---------- Configuration ----------
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
SENDER_EMAIL = "syedaliahmed171@gmail.com"

# ---------- Invoice Data ----------
invoice_data = [
    {
        'customer_name': 'John Doe',
        'customer_email': 'syedaliahmedshah677@gmail.com',
        'date': '2025-03-29',
        'amount': '2500',
        'due_date': '2025-03-10',
        'status': 'unpaid'
    },
    {
        'customer_name': 'Alice Smith',
        'customer_email': 'syedaliahmedshah677@gmail.com',
        'date': '2025-04-15',
        'amount': '1800',
        'due_date': '2025-04-20',
        'status': 'paid'
    }
]

# ---------- Gmail Auth ----------
def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# ---------- PDF Generator ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)
    pdf.output(filename)

# ---------- Send Email ----------
def send_invoice_via_gmail(to_email, subject, body, attachment_path):
    try:
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
        print(f"✅ Email sent to {to_email}")
        return True

    except Exception as e:
        print(f"❌ Failed to send email to {to_email}: {str(e)}")
        return False

# ---------- Full Invoice Process ----------
def full_invoice_process():
    today = datetime.today().strftime('%Y-%m-%d')
    
    for invoice in invoice_data:
        name = invoice['customer_name']
        email = invoice['customer_email']
        status = invoice['status']
        due = invoice['due_date']

        filename = f"{name.replace(' ', '_')}_invoice.pdf"
        create_invoice_pdf(invoice, filename)
        print(f"📄 Invoice created for {name}")

        sent = send_invoice_via_gmail(
            to_email=email,
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path=filename
        )

        if status == 'unpaid' and due < today:
            reminder_file = f"reminder_{name.replace(' ', '_')}.pdf"
            create_invoice_pdf(invoice, reminder_file)
            send_invoice_via_gmail(
                to_email=email,
                subject="Reminder: Invoice Overdue",
                body="Hello, your invoice is overdue. Please pay soon.",
                attachment_path=reminder_file
            )

        # Example paid marker simulation
        if status == 'unpaid' and email == "syedaliahmedshah677@gmail.com":  # Simulate payment for this customer
            invoice['status'] = 'paid'
            print(f"💵 Invoice marked as paid for {email}")

# ---------- Run Main ----------
if __name__ == "__main__":
    full_invoice_process()


📄 Invoice created for John Doe
✅ Email sent to syedaliahmedshah677@gmail.com
✅ Email sent to syedaliahmedshah677@gmail.com
💵 Invoice marked as paid for syedaliahmedshah677@gmail.com
📄 Invoice created for Alice Smith
✅ Email sent to syedaliahmedshah677@gmail.com


In [ ]:
# ---------- Imports ----------
from fpdf import FPDF
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq 
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from google_auth_oauthlib.flow import InstalledAppFlow
from datetime import datetime
import os
import base64

# ---------- Load Environment ----------
load_dotenv()

# ---------- Configuration ----------
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
SENDER_EMAIL = "syedaliahmed171@gmail.com"  # Change this

# ---------- Invoice Data ----------
invoice_data = [
    {
        'customer_name': 'John Doe',
        'customer_email': 'syedaliahmedshah677@gmail.com',
        'date': '2025-04-29',
        'amount': '2500',
        'due_date': '2025-03-10',
        'status': 'unpaid'
    },
    {
        'customer_name': 'Alice Smith',
        'customer_email': 'syedaliahmedshah677@gmail.com',
        'date': '2025-04-15',
        'amount': '1800',
        'due_date': '2025-04-20',
        'status': 'paid'
    }
]

# ---------- Gmail Auth ----------
def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# ---------- PDF Invoice ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)
    pdf.output(filename)

# ---------- Send Email ----------
def send_invoice_via_gmail(to_email, subject, body, attachment_path):
    try:
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
        print(f"✅ Email sent to {to_email}")
        return True
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False

# ---------- Tools ----------
def create_all_invoice_pdfs(_=None, **kwargs):
    for invoice in invoice_data:
        filename = f"invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        create_invoice_pdf(invoice, filename)
    return "✅ All invoices created as PDF files."


def send_all_invoices(_=None, **kwargs):
    responses = []
    for invoice in invoice_data:
        filename = f"invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        if not os.path.exists(filename):
            responses.append(f"❌ PDF not found for {invoice['customer_name']}. Skipping.")
            continue

        success = send_invoice_via_gmail(
            to_email=invoice['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path=filename
        )

        if success:
            responses.append(f"✅ Sent invoice to {invoice['customer_name']}")
        else:
            responses.append(f"❌ Failed to send invoice to {invoice['customer_name']}")

    return "\n".join(responses)



def remind_overdue_invoices(_):
    today = datetime.today().strftime('%Y-%m-%d')
    results = []
    for invoice in invoice_data:
        if invoice['status'] == 'unpaid' and invoice['due_date'] < today:
            filename = f"{invoice['customer_name'].replace(' ', '_')}_reminder.pdf"
            create_invoice_pdf(invoice, filename)
            sent = send_invoice_via_gmail(
                to_email=invoice['customer_email'],
                subject="Payment Reminder",
                body="Reminder for unpaid invoice.",
                attachment_path=filename
            )
            if sent:
                results.append(f"{invoice['customer_name']} ⏰ Reminder sent.")
    return "\n".join(results) if results else "No overdue invoices."

def mark_all_paid(_):
    updated = []
    for invoice in invoice_data:
        if invoice['status'] == 'unpaid':
            invoice['status'] = 'paid'
            updated.append(invoice['customer_name'])
    return "Marked as paid: " + ", ".join(updated) if updated else "No unpaid invoices found."

# ---------- LangChain Agent ----------
tools = [
    Tool(name="CreateInvoices", func=create_all_invoice_pdfs,
         description="Create invoice PDFs for all customers. Input is ignored."),
    Tool(name="SendInvoices", func=send_all_invoices,
         description="Send invoice emails with attached PDFs to all customers."),
    Tool(name="RemindOverdueInvoices", func=remind_overdue_invoices,
         description="Send reminders for overdue unpaid invoices."),
    Tool(name="MarkAllInvoicesPaid", func=mark_all_paid,
         description="Mark all unpaid invoices as paid.")
]


llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.0,
    api_key=GROQ_API_KEY
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate"
)

# ---------- Run Agent Task ----------
print("Starting invoice process...")
task = "Create invoices for all customers. Then send those invoices by email. Then send reminders for overdue unpaid invoices. Finally, mark them as paid if payment is received."
result = agent.run(task)
print("\nResult:\n", result)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Starting invoice process...
Thought: I need to create invoices for all customers, then send them by email, and finally take care of overdue invoices and mark them as paid.

Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Thought: Now that the invoices are created, I need to send them to the customers.

Action: SendInvoices
Action Input: None✅ Email sent to syedaliahmedshah677@gmail.com
✅ Email sent to syedaliahmedshah677@gmail.com

Observation: ✅ Sent invoice to John Doe
✅ Sent invoice to Alice Smith
Thought:Thought: Now that the invoices have been sent, I need to check if there are any overdue invoices and send reminders to the customers who haven't paid yet.

Action: RemindOverdueInvoices
Action Input: None✅ Email sent to syedaliahmedshah677@gmail.com

Observation: John Doe ⏰ Reminder sent.
Thought:Here's the continuation:

Action: None (no action needed, just observation)
Action Input: None
Observation: None (no action needed, just

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def get_invoice_data_from_sheet(sheet_name="Invoices"):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    client = gspread.authorize(creds)

    sheet = client.open(sheet_name).sheet1  # opens the first sheet
    data = sheet.get_all_records()  # returns list of dictionaries
    return data


In [7]:
# ---------- Imports ----------
from fpdf import FPDF
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq 
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from google_auth_oauthlib.flow import InstalledAppFlow
from datetime import datetime
import os
import base64

# ---------- Load Environment ----------
load_dotenv()

# ---------- Configuration ----------
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
SENDER_EMAIL = "syedaliahmed171@gmail.com"  # Change this
SCRIPT_WEB_APP_URL = "https://script.google.com/macros/s/AKfycbzTbvrTcHlLXyRmn_zKVIuehTErVq329a6sBaTgZCcOknzsZyOwoFZ_KGS-HzFgwIMAaw/exec"  # Change this

# ---------- Invoice Data ----------
invoice_data = get_invoice_data_from_sheet()  # You must define this function to fetch from Google Sheets


# ---------- Gmail Auth ----------
def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds


# ---------- PDF Invoice ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)
    pdf.output(filename)


# ---------- Send Email ----------
def send_invoice_via_gmail(to_email, subject, body, attachment_path, html=False):
    try:
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        msg = MIMEMultipart('alternative')
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject

        if html:
            msg.attach(MIMEText(body, 'html'))
        else:
            msg.attach(MIMEText(body, 'plain'))

        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
        print(f"✅ Email sent to {to_email}")
        return True
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False


# ---------- Tools ----------
def create_all_invoice_pdfs(_=None, **kwargs):
    for invoice in invoice_data:
        filename = f"invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        create_invoice_pdf(invoice, filename)
    return "✅ All invoices created as PDF files."


def send_all_invoices(_=None, **kwargs):
    responses = []
    for invoice in invoice_data:
        filename = f"invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        if not os.path.exists(filename):
            responses.append(f"❌ PDF not found for {invoice['customer_name']}. Skipping.")
            continue

        success = send_invoice_via_gmail(
            to_email=invoice['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path=filename
        )

        if success:
            responses.append(f"✅ Sent invoice to {invoice['customer_name']}")
        else:
            responses.append(f"❌ Failed to send invoice to {invoice['customer_name']}")

    return "\n".join(responses)


def remind_overdue_invoices(_=None, **kwargs):
    today = datetime.today().strftime('%Y-%m-%d')
    results = []

    for invoice in invoice_data:
        if invoice['status'] == 'unpaid' and invoice['due_date'] < today:
            filename = f"{invoice['customer_name'].replace(' ', '_')}_reminder.pdf"
            create_invoice_pdf(invoice, filename)

            button_link = f"{SCRIPT_WEB_APP_URL}?customer_name={invoice['customer_name'].replace(' ', '%20')}"
            html_body = f"""
            <html>
                <body>
                    <p>Dear {invoice['customer_name']},</p>
                    <p>This is a reminder for your unpaid invoice.</p>
                    <p><b>Amount:</b> ${invoice['amount']}</p>
                    <p><b>Due Date:</b> {invoice['due_date']}</p>
                    <p>
                        <a href="{button_link}" style="padding: 10px 15px; background-color: #28a745; color: white; text-decoration: none; border-radius: 5px;">
                            Pay Your Invoice Now
                        </a>
                    </p>
                </body>
            </html>
            """

            sent = send_invoice_via_gmail(
                to_email=invoice['customer_email'],
                subject="Payment Reminder",
                body=html_body,
                attachment_path=filename,
                html=True
            )

            if sent:
                results.append(f"{invoice['customer_name']} ⏰ Reminder sent.")

    return "\n".join(results) if results else "No overdue invoices."


def mark_paid_invoices_from_sheet(_=None, **kwargs):
    global invoice_data
    invoice_data = get_invoice_data_from_sheet()

    updated = []
    for invoice in invoice_data:
        if invoice['status'] == 'paid':
            updated.append(f"✅ {invoice['customer_name']}'s invoice is marked as paid.")

    if not updated:
        return "⚠️ No invoices marked as paid yet."

    return "\n".join(updated)


# ---------- LangChain Agent ----------
tools = [
    Tool(name="CreateInvoices", func=create_all_invoice_pdfs,
         description="Create invoice PDFs for all customers. Input is ignored."),
    Tool(name="SendInvoices", func=send_all_invoices,
         description="Send invoice emails with attached PDFs to all customers."),
    Tool(name="RemindOverdueInvoices", func=remind_overdue_invoices,
         description="Send reminders for overdue unpaid invoices."),
    Tool(name="MarkPaidInvoices", func=mark_paid_invoices_from_sheet,
         description="Check the Google Sheet and return list of customers whose invoices were marked as paid after clicking the payment button.")
]

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.0,
    api_key=GROQ_API_KEY
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate"
)

# ---------- Run Agent Task ----------
print("Starting invoice process...")
task = "Create invoices for all customers. Then send those invoices by email. Then send reminders for overdue unpaid invoices. Finally, mark them as paid if payment is received."
result = agent.run(task)
print("\nResult:\n", result)


C:\Users\syeda\AppData\Local\Temp\ipykernel_15192\2947521648.py:190: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(
C:\Users\syeda\AppData\Local\Temp\ipykernel_15192\2947521648.py:203: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = agent.run(task)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Starting invoice process...
Thought: I need to create invoices for all customers, then send them by email, and finally, send reminders for overdue unpaid invoices. After that, I need to mark the paid invoices.

Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Thought: Now that the invoices are created, I need to send them to the customers.

Action: SendInvoices
Action Input: NonePlease visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=582335548132-c3i61idp4ljtrdkm10f1e7ltg1cqmi8i.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51477%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=FfS7Sy3EbSHHsbAbnOF2eZaVd3Jd5i&access_type=offline
✅ Email sent to syedaliahmedshah677@gmail.com
✅ Email sent to syedaliahmedshah677@gmail.com

Observation: ✅ Sent invoice to John Doe
✅ Sent invoice to Alice Smith
Thought:Thought: Now that the invoices have 

In [4]:
# ---------- Imports ----------
import os
import base64
import requests
from datetime import datetime
from fpdf import FPDF
from dotenv import load_dotenv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq

# ---------- Load Environment ----------
load_dotenv()

# ---------- Configuration ----------
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
SENDER_EMAIL = "syedaliahmed171@gmail.com"  # Change this to your email



# ---------- Global Invoice Data ----------
invoice_data = get_invoice_data_from_sheet()

# ---------- Gmail Auth ----------
def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# ---------- PDF Invoice ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)
    pdf.output(filename)

# ---------- Send Email ----------
def send_invoice_via_gmail(to_email, subject, body, attachment_path,is_html=False):
    try:
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'html' if is_html else 'plain'))

        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
        print(f"✅ Email sent to {to_email}")
        return True
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False

# ---------- Tools ----------
def create_all_invoice_pdfs(_=None, **kwargs):
    for invoice in invoice_data:
        filename = f"invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        create_invoice_pdf(invoice, filename)
    return "✅ All invoices created as PDF files."

def send_all_invoices(_=None, **kwargs):
    responses = []
    for invoice in invoice_data:
        filename = f"invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        if not os.path.exists(filename):
            responses.append(f"❌ PDF not found for {invoice['customer_name']}. Skipping.")
            continue
        success = send_invoice_via_gmail(
            to_email=invoice['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path=filename
        )
        if success:
            responses.append(f"✅ Sent invoice to {invoice['customer_name']}")
        else:
            responses.append(f"❌ Failed to send invoice to {invoice['customer_name']}")
    return "\n".join(responses)

def remind_overdue_invoices(_):
    today = datetime.today().strftime('%Y-%m-%d')
    results = []
    
    for invoice in invoice_data:
        if invoice['status'] == 'unpaid' and invoice['due_date'] < today:
            filename = f"{invoice['customer_name'].replace(' ', '_')}_reminder.pdf"
            create_invoice_pdf(invoice, filename)

            # Generate Pay Now URL
            pay_now_url = f"https://script.google.com/macros/s/AKfycbwmJ-cxnBxqtrbM0h3xobFvQ3nU9ATKhYPHflBx7fJcJTGtT2Hh3nZjwZNa28tC5b0W/exec?invoice_id={invoice['invoice_id']}"

            # Compose HTML email body
            html_body = f"""
            <p>Dear {invoice['customer_name']},</p>
            <p>This is a reminder that your invoice dated <b>{invoice['date']}</b> is overdue.</p>
            <p>Amount Due: <b>${invoice['amount']}</b></p>
            <p>Please click the button below to mark your invoice as paid:</p>
            <a href="{pay_now_url}" style="background-color:#28a745;color:white;padding:10px 15px;text-decoration:none;border-radius:5px;">✅ Pay Now</a>
            <p>Thank you!</p>
            """

            sent = send_invoice_via_gmail(
                to_email=invoice['customer_email'],
                subject="⏰ Payment Reminder - Invoice Overdue",
                body=html_body,
                attachment_path=filename,
                is_html=True  # enable HTML content
            )

            if sent:
                results.append(f"{invoice['customer_name']} ⏰ Reminder sent.")

    return "\n".join(results) if results else "No overdue invoices."


def mark_paid_invoices_from_sheet(_=None, **kwargs):
    global invoice_data
    invoice_data = get_invoice_data_from_sheet()

    updated = []
    for invoice in invoice_data:
        if invoice['status'] == 'paid':
            updated.append(f"✅ {invoice['customer_name']}'s invoice is marked as paid.")

    if not updated:
        return "⚠️ No invoices marked as paid yet."


    return "\n".join(updated)

# ---------- LangChain Agent ----------
tools = [
    Tool(name="CreateInvoices", func=create_all_invoice_pdfs,
         description="Create invoice PDFs for all customers."),
    Tool(name="SendInvoices", func=send_all_invoices,
         description="Send invoice emails with attached PDFs to all customers."),
    Tool(name="RemindOverdueInvoices", func=remind_overdue_invoices,
         description="Send reminders for overdue unpaid invoices."),
    Tool(name="MarkPaidInvoices", func=mark_paid_invoices_from_sheet,
         description="Mark paid invoices and trigger the Web App for unpaid ones.")
]

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.0,
    api_key=GROQ_API_KEY
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate"
)

# ---------- Run Agent Task ----------
print("Starting invoice process...")
task = (
    "Create invoices for all customers. "
    "Then send those invoices by email. "
    "Then send reminders for overdue unpaid invoices. "
    "Finally, mark them as paid if payment is received."
)
result = agent.run(task)
print("\nResult:\n", result)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Starting invoice process...
Thought: I need to create invoices for all customers, then send them by email, and finally handle any overdue invoices and mark paid invoices.

Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Thought: Now that the invoices are created, I need to send them to the customers.

Action: SendInvoices
Action Input: None✅ Email sent to syedaliahmedshah677@gmail.com
✅ Email sent to syedaliahmedshah677@gmail.com

Observation: ✅ Sent invoice to John Doe
✅ Sent invoice to Alice Smith
Thought:Thought: Now that the invoices have been sent, I need to check if there are any overdue invoices and send reminders to the customers.

Action: RemindOverdueInvoices
Action Input: None✅ Email sent to syedaliahmedshah677@gmail.com

Observation: John Doe ⏰ Reminder sent.
Thought:Thought: Now that the reminders have been sent, I need to check if any payments have been received and mark the corresponding invoices as paid.

Action: MarkP

In [ ]:
import os
import json
import base64
import gspread
from fpdf import FPDF
from datetime import datetime
from dotenv import load_dotenv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq

# ---------- Load Environment ----------
load_dotenv()
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SPREADSHEET_NAME = "Invoices"
SENDER_EMAIL = "syedaliahmed171@gmail.com"  # change to yours

# ---------- Google Sheets Client ----------
def get_gsheet_client():
    scope = ["https://spreadsheets.google.com/feeds", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    return gspread.authorize(creds)



# ---------- Gmail Auth ----------
def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds


# ---------- Complete Tool Set ----------
def fetch_payroll_data_tool(_=None):
    """Fetch all payroll data from Google Sheets"""
    try:
        client = get_gsheet_client()
        spreadsheet = client.open(SPREADSHEET_NAME)
        
        data = {
            "employees": spreadsheet.worksheet("Employees").get_all_records(),
            "attendance": spreadsheet.worksheet("Attendance").get_all_records(),
            "policy": spreadsheet.worksheet("SalaryPolicy").get_all_records()
        }
        return json.dumps({"status": "success", "data": data})
    except Exception as e:
        return json.dumps({"status": "error", "message": f"Error fetching data: {str(e)}"})
    
    
def calculate_salaries_tool(_=None):
    """Calculate all employee salaries"""
    try:
        fetch_result = json.loads(fetch_payroll_data_tool())
        if fetch_result["status"] != "success":
            return fetch_result["message"]
            
        data = fetch_result["data"]
        employees = data["employees"]
        attendance = data["attendance"]
        policy_map = {p["rule_name"]: int(p["value"]) for p in data["policy"]}
        
        results = []
        for emp in employees:
            emp_id = emp["employee_id"]
            att = next((a for a in attendance if a["employee_id"] == emp_id), None)
            if not att:
                continue

            # Modified salary handling:
            base = emp["base_salary"]
            if isinstance(base, str):
                base = int(base.replace(",", ""))
            else:
                base = int(base)
            department = emp.get("department", "Unassigned")
            leaves = int(att.get("leaves_taken", 0))
            allowed = int(att.get("allowed_leaves", 0))
            late = int(att.get("late_arrivals", 0))
            overtime = int(att.get("overtime_hours", 0))

            extra_leaves = max(0, leaves - allowed)
            deductions = (extra_leaves * policy_map.get("leave_penalty", 0)) + \
                         (late * policy_map.get("late_penalty", 0))
            bonus = min(overtime, policy_map.get("max_overtime_allowed", 20)) * \
                    policy_map.get("overtime_rate", 0)
            net = base - deductions + bonus

            results.append({
                "employee_id": emp_id,
                "name": emp["name"],
                "email": emp["email"],
                "base_salary": base,
                "deductions": deductions,
                "bonus": bonus,
                "net_salary": net,
                "department": department

            })
            
        return json.dumps({"status": "success", "data": results})
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})




def generate_payslips_tool(_=None, **kwargs):
    """Generate PDF payslips for all employees using pre-calculated salary data and avoid duplicate sheet entries"""
    try:
        # Get pre-calculated salary data
        calc_result = json.loads(calculate_salaries_tool())
        if calc_result["status"] != "success":
            return calc_result["message"]
            
        employees = calc_result["data"]
        
        # Create payslips directory if not exists
        os.makedirs("payslips", exist_ok=True)
        
        # GSheet client for saving records
        client = get_gsheet_client()
        payslip_sheet = client.open(SPREADSHEET_NAME).worksheet("Payslips")
        existing_records = payslip_sheet.get_all_records()

        today_date = datetime.now().strftime('%Y-%m-%d')
        current_month = datetime.now().strftime('%Y-%m')
        results = []

        for emp in employees:
            emp_id = emp["employee_id"]

            # Generate PDF regardless of duplication
            filename = f"payslips/payslip_{emp_id}.pdf"
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)

            pdf.cell(200, 10, txt="MONTHLY PAYSLIP", ln=True, align='C')
            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Employee: {emp['name']} ({emp_id})", ln=True)
            pdf.cell(200, 10, txt=f"Department: {emp['department']}", ln=True)
            
            pdf.cell(200, 10, txt=f"Period: {datetime.now().strftime('%B %Y')}", ln=True)
            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Base Salary: ${emp['base_salary']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Deductions: ${emp['deductions']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Bonus: ${emp['bonus']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Net Salary: ${emp['net_salary']:,}", ln=True, border=True)
            pdf.output(filename)

            # ✅ Only append to sheet if not already added for this month
            if not any(
                r.get("employee_id") == emp_id and r.get("month", "").startswith(current_month)
                for r in existing_records
            ):
                payslip_sheet.append_row([
                    emp_id,
                    emp["name"],
                    emp["net_salary"],
                    current_month
                ])
                results.append(f"✅ Payslip recorded for {emp['name']}")
            else:
                results.append(f"⚠️ Payslip already recorded for {emp['name']}")

        return json.dumps({
    "status": "success", 
    "message": "Payslips ready. NEXT STEP: Call SendPayslips to email them.",
    "completion_flag": "ready_for_email"  # New explicit signal
})

    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})






def send_payslips_tool(_=None, **kwargs):
    """Send payslip emails to all employees in batches"""
    try:
        # Get payroll data for emails
        fetch_result = json.loads(fetch_payroll_data_tool())
        if fetch_result["status"] != "success":
            return fetch_result["message"]

        employees = fetch_result["data"]["employees"]

        # Check payslips exist
        payslip_files = [f for f in os.listdir("payslips") if f.startswith("payslip_")]
        if not payslip_files:
            return "⚠️ No payslips found. Generate them first."

        # Gmail setup
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        results = []
        batch_size = 10
        for i in range(0, len(employees), batch_size):
            batch = employees[i:i + batch_size]
            for emp in batch:
                filename = f"payslips/payslip_{emp['employee_id']}.pdf"
                if not os.path.exists(filename):
                    results.append(f"⚠️ Payslip not found for {emp['name']}")
                    continue

                msg = MIMEMultipart()
                msg['From'] = SENDER_EMAIL
                msg['To'] = emp['email']
                msg['Subject'] = f"Your Payslip - {datetime.now().strftime('%B %Y')}"

                body = f"""Dear {emp['name']},

Please find attached your payslip for {datetime.now().strftime('%B %Y')}.

Details:
* Employee ID: {emp['employee_id']}
* Department: {emp['department']}

If you have any questions, please contact HR.

Best regards,
Payroll Department
"""
                msg.attach(MIMEText(body, 'plain'))

                with open(filename, "rb") as f:
                    part = MIMEApplication(f.read(), Name=os.path.basename(filename))
                part['Content-Disposition'] = f'attachment; filename="{os.path.basename(filename)}"'
                msg.attach(part)

                raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
                service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
                results.append(f"✅ Sent to {emp['name']} ({emp['email']})")

        return "📤 Email sending results:\n" + "\n".join(results)

    except Exception as e:
        return f"❌ Error sending payslips: {str(e)}"

# ---------- Complete Tool Definitions ----------
tools = [
    Tool(
        name="FetchPayrollData",
        func=fetch_payroll_data_tool,
        description="Fetches all payroll data from Google Sheets. Returns JSON with employees, attendance, and policy data."
    ),
    Tool(
        name="CalculateSalaries",
        func=calculate_salaries_tool,
        description="Calculates net salaries for all employees. Returns JSON results."
    ),
    Tool(
        name="GeneratePayslips",
        func=generate_payslips_tool,
        description="Generates PDF payslips for all employees. Returns status message."
    ),
    Tool(
        name="SendPayslips",
        func=send_payslips_tool,
        description="Sends generated payslips to employees. Returns status message."
    )
]

# ---------- Agent Setup ----------
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    api_key=GROQ_API_KEY
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=7,

)

# ---------- Main Execution ----------
if __name__ == "__main__":
    print("🚀 Starting Payroll Processing Agent...")
    task = """Please complete ALL of these steps in order:
    1. Fetch the latest payroll data
    2. Calculate all employee salaries
    3. Generate PDF payslips for everyone
    4. Send ALL payslips via email
    """
    result = agent.run(task)
    print("\nFinal Result:", result)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


🚀 Starting Payroll Processing Agent...
Thought: I need to complete the payroll processing steps in order to ensure accurate and timely payment to employees. I'll start by fetching the latest payroll data to get the necessary information.

Action: FetchPayrollData
Action Input: None
Observation: {"status": "success", "data": {"employees": [{"employee_id": "EMP001", "name": "Ali Ahmed", "email": "syedaliahmedshah677@gmail.com", "base_salary": 80000, "loan_balance": 15000, "department": "Finance"}, {"employee_id": "EMP002", "name": "Sana Malik", "email": "syedaliahmedshah677@gmail.com", "base_salary": 60000, "loan_balance": 0, "department": "Marketing"}, {"employee_id": "EMP003", "name": "Omar Khan", "email": "omar@company.com", "base_salary": 75000, "loan_balance": 8000, "department": "IT"}, {"employee_id": "EMP004", "name": "Fatima Zahra", "email": "fatima@company.com", "base_salary": 65000, "loan_balance": 12000, "department": "HR"}, {"employee_id": "EMP005", "name": "Bilal Hussain", "

In [ ]:
import os
import json
import base64
import gspread
import requests
from fpdf import FPDF
from datetime import datetime
from dotenv import load_dotenv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_groq import ChatGroq
from typing import Dict, List, Optional

# ---------- Load Environment ----------
load_dotenv()
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SPREADSHEET_NAME = "Invoices"
SENDER_EMAIL = "syedaliahmed171@gmail.com"  # change to yours

# ---------- Shared Data Structures ----------
class SharedData:
    def __init__(self):
        self.payroll_data = None
        self.invoice_data = None
        self.last_execution = {}

shared_data = SharedData()

# ---------- Shared Utility Functions ----------
def get_gsheet_client():
    scope = ["https://spreadsheets.google.com/feeds", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    return gspread.authorize(creds)

def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def get_invoice_data_from_sheet():
    try:
        client = get_gsheet_client()
        spreadsheet = client.open(SPREADSHEET_NAME)
        worksheet = spreadsheet.worksheet("Invoices")
        return worksheet.get_all_records()
    except Exception as e:
        print(f"Error fetching invoice data: {e}")
        return []

# ---------- Payroll Agent Tools ----------
def fetch_payroll_data_tool(_=None):
    """Fetch all payroll data from Google Sheets"""
    try:
        client = get_gsheet_client()
        spreadsheet = client.open(SPREADSHEET_NAME)
        
        data = {
            "employees": spreadsheet.worksheet("Employees").get_all_records(),
            "attendance": spreadsheet.worksheet("Attendance").get_all_records(),
            "policy": spreadsheet.worksheet("SalaryPolicy").get_all_records()
        }
        shared_data.payroll_data = data
        return json.dumps({"status": "success", "data": data})
    except Exception as e:
        return json.dumps({"status": "error", "message": f"Error fetching data: {str(e)}"})

def calculate_salaries_tool(_=None):
    """Calculate all employee salaries"""
    try:
        if shared_data.payroll_data is None:
            fetch_result = json.loads(fetch_payroll_data_tool())
            if fetch_result["status"] != "success":
                return fetch_result["message"]
            data = fetch_result["data"]
        else:
            data = shared_data.payroll_data
            
        employees = data["employees"]
        attendance = data["attendance"]
        policy_map = {p["rule_name"]: int(p["value"]) for p in data["policy"]}
        
        results = []
        for emp in employees:
            emp_id = emp["employee_id"]
            att = next((a for a in attendance if a["employee_id"] == emp_id), None)
            if not att:
                continue

            base = emp["base_salary"]
            if isinstance(base, str):
                base = int(base.replace(",", ""))
            else:
                base = int(base)
            department = emp.get("department", "Unassigned")
            leaves = int(att.get("leaves_taken", 0))
            allowed = int(att.get("allowed_leaves", 0))
            late = int(att.get("late_arrivals", 0))
            overtime = int(att.get("overtime_hours", 0))

            extra_leaves = max(0, leaves - allowed)
            deductions = (extra_leaves * policy_map.get("leave_penalty", 0)) + \
                         (late * policy_map.get("late_penalty", 0))
            bonus = min(overtime, policy_map.get("max_overtime_allowed", 20)) * \
                    policy_map.get("overtime_rate", 0)
            net = base - deductions + bonus

            results.append({
                "employee_id": emp_id,
                "name": emp["name"],
                "email": emp["email"],
                "base_salary": base,
                "deductions": deductions,
                "bonus": bonus,
                "net_salary": net,
                "department": department
            })
            
        return json.dumps({"status": "success", "data": results})
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

def generate_payslips_tool(_=None, **kwargs):
    """Generate PDF payslips for all employees"""
    try:
        calc_result = json.loads(calculate_salaries_tool())
        if calc_result["status"] != "success":
            return calc_result["message"]
            
        employees = calc_result["data"]
        os.makedirs("payslips", exist_ok=True)
        
        client = get_gsheet_client()
        payslip_sheet = client.open(SPREADSHEET_NAME).worksheet("Payslips")
        existing_records = payslip_sheet.get_all_records()

        today_date = datetime.now().strftime('%Y-%m-%d')
        current_month = datetime.now().strftime('%Y-%m')
        results = []

        for emp in employees:
            emp_id = emp["employee_id"]
            filename = f"payslips/payslip_{emp_id}.pdf"
            
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.cell(200, 10, txt="MONTHLY PAYSLIP", ln=True, align='C')
            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Employee: {emp['name']} ({emp_id})", ln=True)
            pdf.cell(200, 10, txt=f"Department: {emp['department']}", ln=True)
            pdf.cell(200, 10, txt=f"Period: {datetime.now().strftime('%B %Y')}", ln=True)
            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Base Salary: ${emp['base_salary']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Deductions: ${emp['deductions']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Bonus: ${emp['bonus']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Net Salary: ${emp['net_salary']:,}", ln=True, border=True)
            pdf.output(filename)

            if not any(
                r.get("employee_id") == emp_id and r.get("month", "").startswith(current_month)
                for r in existing_records
            ):
                payslip_sheet.append_row([
                    emp_id,
                    emp["name"],
                    emp["net_salary"],
                    current_month
                ])
                results.append(f"✅ Payslip recorded for {emp['name']}")
            else:
                results.append(f"⚠️ Payslip already recorded for {emp['name']}")

        return json.dumps({
            "status": "success", 
            "message": "Payslips ready. NEXT STEP: Call SendPayslips to email them.",
            "completion_flag": "ready_for_email"
        })
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

def send_payslips_tool(_=None, **kwargs):
    """Send payslip emails to all employees"""
    try:
        if shared_data.payroll_data is None:
            fetch_result = json.loads(fetch_payroll_data_tool())
            if fetch_result["status"] != "success":
                return fetch_result["message"]
            employees = fetch_result["data"]["employees"]
        else:
            employees = shared_data.payroll_data["employees"]

        payslip_files = [f for f in os.listdir("payslips") if f.startswith("payslip_")]
        if not payslip_files:
            return "⚠️ No payslips found. Generate them first."

        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)
        results = []
        
        for emp in employees:
            filename = f"payslips/payslip_{emp['employee_id']}.pdf"
            if not os.path.exists(filename):
                results.append(f"⚠️ Payslip not found for {emp['name']}")
                continue

            msg = MIMEMultipart()
            msg['From'] = SENDER_EMAIL
            msg['To'] = emp['email']
            msg['Subject'] = f"Your Payslip - {datetime.now().strftime('%B %Y')}"

            body = f"""Dear {emp['name']},
Please find attached your payslip for {datetime.now().strftime('%B %Y')}.

Details:
* Employee ID: {emp['employee_id']}
* Department: {emp['department']}

If you have any questions, please contact HR.

Best regards,
Payroll Department
"""
            msg.attach(MIMEText(body, 'plain'))
            with open(filename, "rb") as f:
                part = MIMEApplication(f.read(), Name=os.path.basename(filename))
            part['Content-Disposition'] = f'attachment; filename="{os.path.basename(filename)}"'
            msg.attach(part)

            raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
            service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
            results.append(f"✅ Sent to {emp['name']} ({emp['email']})")

        return "📤 Email sending results:\n" + "\n".join(results)
    except Exception as e:
        return f"❌ Error sending payslips: {str(e)}"

# ---------- Invoice Agent Tools ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)
    pdf.output(filename)

def send_invoice_via_gmail(to_email, subject, body, attachment_path, is_html=False):
    try:
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'html' if is_html else 'plain'))

        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
        return True
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return False

def create_all_invoice_pdfs_tool(_=None, **kwargs):
    """Create invoice PDFs for all customers"""
    shared_data.invoice_data = get_invoice_data_from_sheet()
    for invoice in shared_data.invoice_data:
        filename = f"invoices/invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        os.makedirs("invoices", exist_ok=True)
        create_invoice_pdf(invoice, filename)
    return "✅ All invoices created as PDF files."

def send_all_invoices_tool(_=None, **kwargs):
    """Send invoice emails with attached PDFs to all customers"""
    if shared_data.invoice_data is None:
        shared_data.invoice_data = get_invoice_data_from_sheet()
        
    responses = []
    for invoice in shared_data.invoice_data:
        filename = f"invoices/invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        if not os.path.exists(filename):
            responses.append(f"❌ PDF not found for {invoice['customer_name']}. Skipping.")
            continue
            
        success = send_invoice_via_gmail(
            to_email=invoice['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path=filename
        )
        if success:
            responses.append(f"✅ Sent invoice to {invoice['customer_name']}")
        else:
            responses.append(f"❌ Failed to send invoice to {invoice['customer_name']}")
    return "\n".join(responses)

def remind_overdue_invoices_tool(_=None, **kwargs):
    """Send reminders for overdue unpaid invoices"""
    if shared_data.invoice_data is None:
        shared_data.invoice_data = get_invoice_data_from_sheet()
        
    today = datetime.today().strftime('%Y-%m-%d')
    results = []
    
    for invoice in shared_data.invoice_data:
        if invoice['status'] == 'unpaid' and invoice['due_date'] < today:
            filename = f"invoices/{invoice['customer_name'].replace(' ', '_')}_reminder.pdf"
            create_invoice_pdf(invoice, filename)

            pay_now_url = f"https://script.google.com/macros/s/AKfycbwmJ-cxnBxqtrbM0h3xobFvQ3nU9ATKhYPHflBx7fJcJTGtT2Hh3nZjwZNa28tC5b0W/exec?invoice_id={invoice['invoice_id']}"

            html_body = f"""
            <p>Dear {invoice['customer_name']},</p>
            <p>This is a reminder that your invoice dated <b>{invoice['date']}</b> is overdue.</p>
            <p>Amount Due: <b>${invoice['amount']}</b></p>
            <p>Please click the button below to mark your invoice as paid:</p>
            <a href="{pay_now_url}" style="background-color:#28a745;color:white;padding:10px 15px;text-decoration:none;border-radius:5px;">✅ Pay Now</a>
            <p>Thank you!</p>
            """

            sent = send_invoice_via_gmail(
                to_email=invoice['customer_email'],
                subject="⏰ Payment Reminder - Invoice Overdue",
                body=html_body,
                attachment_path=filename,
                is_html=True
            )

            if sent:
                results.append(f"{invoice['customer_name']} ⏰ Reminder sent.")

    return "\n".join(results) if results else "No overdue invoices."

def mark_paid_invoices_tool(_=None, **kwargs):
    """Mark paid invoices from sheet"""
    shared_data.invoice_data = get_invoice_data_from_sheet()
    updated = []
    for invoice in shared_data.invoice_data:
        if invoice['status'] == 'paid':
            updated.append(f"✅ {invoice['customer_name']}'s invoice is marked as paid.")
    return "\n".join(updated) if updated else "⚠️ No invoices marked as paid yet."

# ---------- Agent Coordination ----------
def coordinator_agent(query: str):
    """Determine which agent should handle the request"""
    payroll_keywords = ["payroll", "salary", "payslip", "employee payment"]
    invoice_keywords = ["invoice", "customer payment", "billing", "reminder"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in payroll_keywords):
        return "payroll"
    elif any(keyword in query_lower for keyword in invoice_keywords):
        return "invoice"
    else:
        return "both"

# ---------- Agent Initialization ----------
payroll_tools = [
    Tool(name="FetchPayrollData", func=fetch_payroll_data_tool,
         description="Fetches payroll data from Google Sheets"),
    Tool(name="CalculateSalaries", func=calculate_salaries_tool,
         description="Calculates net salaries for all employees"),
    Tool(name="GeneratePayslips", func=generate_payslips_tool,
         description="Generates PDF payslips for all employees"),
    Tool(name="SendPayslips", func=send_payslips_tool,
         description="Sends generated payslips to employees")
]

invoice_tools = [
    Tool(name="CreateInvoices", func=create_all_invoice_pdfs_tool,
         description="Create invoice PDFs for all customers"),
    Tool(name="SendInvoices", func=send_all_invoices_tool,
         description="Send invoice emails to all customers"),
    Tool(name="RemindOverdueInvoices", func=remind_overdue_invoices_tool,
         description="Send reminders for overdue unpaid invoices"),
    Tool(name="MarkPaidInvoices", func=mark_paid_invoices_tool,
         description="Mark paid invoices from sheet")
]

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    api_key=GROQ_API_KEY
)

payroll_agent = initialize_agent(
    payroll_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=7
)

invoice_agent = initialize_agent(
    invoice_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

# ---------- Main Execution ----------
def execute_task(task: str):
    agent_type = coordinator_agent(task)
    
    if agent_type == "payroll":
        print("\n🚀 Payroll Agent Processing...")
        return payroll_agent.run(task)
    elif agent_type == "invoice":
        print("\n🚀 Invoice Agent Processing...")
        return invoice_agent.run(task)
    else:
        print("\n🚀 Both Agents Processing...")
        payroll_result = payroll_agent.run(task)
        invoice_result = invoice_agent.run(task)
        return f"Payroll Results:\n{payroll_result}\n\nInvoice Results:\n{invoice_result}"

if __name__ == "__main__":
    print("Multi-Agent System Ready")
    while True:
        try:
            task = input("\nEnter your task (or 'quit' to exit): ")
            if task.lower() == 'quit':
                break
                
            result = execute_task(task)
            print("\nResult:\n", result)
            
        except Exception as e:
            print(f"Error: {str(e)}")






# payslips:"Please complete ALL of these steps in order:
#     1. Fetch the latest payroll data
#     2. Calculate all employee salaries
#     3. Generate PDF payslips for everyone
#     4. Send ALL payslips via email

# invoice:  "Create invoices for all customers, Then send those invoices by email, Then send reminders for overdue unpaid invoices, Finally, mark them as paid if payment is received."

Multi-Agent System Ready


C:\Users\syeda\AppData\Local\Temp\ipykernel_26208\2479450120.py:437: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  return invoice_agent.run(task)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



🚀 Invoice Agent Processing...
Question: invoice
Thought: I need to create an invoice for a customer.
Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Here's the response:

Question: invoice
Thought: I need to create an invoice for a customer.
Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Let's continue!

Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Let's continue!

Action: SendInvoices
Action Input: NonePlease visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=582335548132-c3i61idp4ljtrdkm10f1e7ltg1cqmi8i.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52982%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=aofPa0odzUb3zUJywpBItGqqETtf9i&access_type=offline

Observation: ✅ Sent invoice to John Doe
✅ Sent invoice to Alice Smith
T

In [5]:
import os
import json
import base64
import gspread
import requests
from fpdf import FPDF
from datetime import datetime
from dotenv import load_dotenv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from langchain.agents import Tool, initialize_agent, AgentType, AgentExecutor
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from typing import List, Union, Dict, Optional, Type
from langchain.agents.agent import Agent, AgentOutputParser
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

# ---------- Load Environment ----------
load_dotenv()
CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SPREADSHEET_NAME = "Invoices"
SENDER_EMAIL = "syedaliahmed171@gmail.com"  # change to yours

# ---------- Shared Data Structures ----------
class SharedData:
    def __init__(self):
        self.payroll_data = None
        self.invoice_data = None
        self.last_execution = {}

shared_data = SharedData()

# ---------- Shared Utility Functions ----------
def get_gsheet_client():
    scope = ["https://spreadsheets.google.com/feeds", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    return gspread.authorize(creds)

def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def get_invoice_data_from_sheet():
    try:
        client = get_gsheet_client()
        spreadsheet = client.open(SPREADSHEET_NAME)
        worksheet = spreadsheet.worksheet("Invoices")
        return worksheet.get_all_records()
    except Exception as e:
        print(f"Error fetching invoice data: {e}")
        return []

# ---------- Payroll Agent Tools ----------
def fetch_payroll_data_tool(_=None):
    """Fetch all payroll data from Google Sheets"""
    try:
        client = get_gsheet_client()
        spreadsheet = client.open(SPREADSHEET_NAME)
        
        data = {
            "employees": spreadsheet.worksheet("Employees").get_all_records(),
            "attendance": spreadsheet.worksheet("Attendance").get_all_records(),
            "policy": spreadsheet.worksheet("SalaryPolicy").get_all_records()
        }
        shared_data.payroll_data = data
        return json.dumps({"status": "success", "data": data})
    except Exception as e:
        return json.dumps({"status": "error", "message": f"Error fetching data: {str(e)}"})

def calculate_salaries_tool(_=None):
    """Calculate all employee salaries"""
    try:
        if shared_data.payroll_data is None:
            fetch_result = json.loads(fetch_payroll_data_tool())
            if fetch_result["status"] != "success":
                return fetch_result["message"]
            data = fetch_result["data"]
        else:
            data = shared_data.payroll_data
            
        employees = data["employees"]
        attendance = data["attendance"]
        policy_map = {p["rule_name"]: int(p["value"]) for p in data["policy"]}
        
        results = []
        for emp in employees:
            emp_id = emp["employee_id"]
            att = next((a for a in attendance if a["employee_id"] == emp_id), None)
            if not att:
                continue

            base = emp["base_salary"]
            if isinstance(base, str):
                base = int(base.replace(",", ""))
            else:
                base = int(base)
            department = emp.get("department", "Unassigned")
            leaves = int(att.get("leaves_taken", 0))
            allowed = int(att.get("allowed_leaves", 0))
            late = int(att.get("late_arrivals", 0))
            overtime = int(att.get("overtime_hours", 0))

            extra_leaves = max(0, leaves - allowed)
            deductions = (extra_leaves * policy_map.get("leave_penalty", 0)) + \
                         (late * policy_map.get("late_penalty", 0))
            bonus = min(overtime, policy_map.get("max_overtime_allowed", 20)) * \
                    policy_map.get("overtime_rate", 0)
            net = base - deductions + bonus

            results.append({
                "employee_id": emp_id,
                "name": emp["name"],
                "email": emp["email"],
                "base_salary": base,
                "deductions": deductions,
                "bonus": bonus,
                "net_salary": net,
                "department": department
            })
            
        return json.dumps({"status": "success", "data": results})
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

def generate_payslips_tool(_=None, **kwargs):
    """Generate PDF payslips for all employees"""
    try:
        calc_result = json.loads(calculate_salaries_tool())
        if calc_result["status"] != "success":
            return calc_result["message"]
            
        employees = calc_result["data"]
        os.makedirs("payslips", exist_ok=True)
        
        client = get_gsheet_client()
        payslip_sheet = client.open(SPREADSHEET_NAME).worksheet("Payslips")
        existing_records = payslip_sheet.get_all_records()

        today_date = datetime.now().strftime('%Y-%m-%d')
        current_month = datetime.now().strftime('%Y-%m')
        results = []

        for emp in employees:
            emp_id = emp["employee_id"]
            filename = f"payslips/payslip_{emp_id}.pdf"
            
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.cell(200, 10, txt="MONTHLY PAYSLIP", ln=True, align='C')
            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Employee: {emp['name']} ({emp_id})", ln=True)
            pdf.cell(200, 10, txt=f"Department: {emp['department']}", ln=True)
            pdf.cell(200, 10, txt=f"Period: {datetime.now().strftime('%B %Y')}", ln=True)
            pdf.ln(10)
            pdf.cell(200, 10, txt=f"Base Salary: ${emp['base_salary']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Deductions: ${emp['deductions']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Bonus: ${emp['bonus']:,}", ln=True)
            pdf.cell(200, 10, txt=f"Net Salary: ${emp['net_salary']:,}", ln=True, border=True)
            pdf.output(filename)

            if not any(
                r.get("employee_id") == emp_id and r.get("month", "").startswith(current_month)
                for r in existing_records
            ):
                payslip_sheet.append_row([
                    emp_id,
                    emp["name"],
                    emp["net_salary"],
                    current_month
                ])
                results.append(f"✅ Payslip recorded for {emp['name']}")
            else:
                results.append(f"⚠️ Payslip already recorded for {emp['name']}")

        return json.dumps({
            "status": "success", 
            "message": "Payslips ready. NEXT STEP: Call SendPayslips to email them.",
            "completion_flag": "ready_for_email"
        })
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

def send_payslips_tool(_=None, **kwargs):
    """Send payslip emails to all employees"""
    try:
        if shared_data.payroll_data is None:
            fetch_result = json.loads(fetch_payroll_data_tool())
            if fetch_result["status"] != "success":
                return fetch_result["message"]
            employees = fetch_result["data"]["employees"]
        else:
            employees = shared_data.payroll_data["employees"]

        payslip_files = [f for f in os.listdir("payslips") if f.startswith("payslip_")]
        if not payslip_files:
            return "⚠️ No payslips found. Generate them first."

        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)
        results = []
        
        for emp in employees:
            filename = f"payslips/payslip_{emp['employee_id']}.pdf"
            if not os.path.exists(filename):
                results.append(f"⚠️ Payslip not found for {emp['name']}")
                continue

            msg = MIMEMultipart()
            msg['From'] = SENDER_EMAIL
            msg['To'] = emp['email']
            msg['Subject'] = f"Your Payslip - {datetime.now().strftime('%B %Y')}"

            body = f"""Dear {emp['name']},
Please find attached your payslip for {datetime.now().strftime('%B %Y')}.

Details:
* Employee ID: {emp['employee_id']}
* Department: {emp['department']}

If you have any questions, please contact HR.

Best regards,
Payroll Department
"""
            msg.attach(MIMEText(body, 'plain'))
            with open(filename, "rb") as f:
                part = MIMEApplication(f.read(), Name=os.path.basename(filename))
            part['Content-Disposition'] = f'attachment; filename="{os.path.basename(filename)}"'
            msg.attach(part)

            raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
            service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
            results.append(f"✅ Sent to {emp['name']} ({emp['email']})")

        return "📤 Email sending results:\n" + "\n".join(results)
    except Exception as e:
        return f"❌ Error sending payslips: {str(e)}"

# ---------- Invoice Agent Tools ----------
def create_invoice_pdf(data, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="INVOICE", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Customer: {data['customer_name']}", ln=True)
    pdf.cell(200, 10, txt=f"Date: {data['date']}", ln=True)
    pdf.cell(200, 10, txt=f"Amount: ${data['amount']}", ln=True)
    pdf.cell(200, 10, txt=f"Due Date: {data['due_date']}", ln=True)
    pdf.output(filename)

def send_invoice_via_gmail(to_email, subject, body, attachment_path, is_html=False):
    try:
        creds = get_gmail_credentials()
        service = build('gmail', 'v1', credentials=creds)

        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'html' if is_html else 'plain'))

        with open(attachment_path, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment_path)}"'
        msg.attach(part)

        raw_msg = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        service.users().messages().send(userId="me", body={"raw": raw_msg}).execute()
        return True
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return False

def create_all_invoice_pdfs_tool(_=None, **kwargs):
    """Create invoice PDFs for all customers"""
    shared_data.invoice_data = get_invoice_data_from_sheet()
    for invoice in shared_data.invoice_data:
        filename = f"invoices/invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        os.makedirs("invoices", exist_ok=True)
        create_invoice_pdf(invoice, filename)
    return "✅ All invoices created as PDF files."

def send_all_invoices_tool(_=None, **kwargs):
    """Send invoice emails with attached PDFs to all customers"""
    if shared_data.invoice_data is None:
        shared_data.invoice_data = get_invoice_data_from_sheet()
        
    responses = []
    for invoice in shared_data.invoice_data:
        filename = f"invoices/invoice_{invoice['customer_name'].replace(' ', '_')}.pdf"
        if not os.path.exists(filename):
            responses.append(f"❌ PDF not found for {invoice['customer_name']}. Skipping.")
            continue
            
        success = send_invoice_via_gmail(
            to_email=invoice['customer_email'],
            subject="Your Invoice from Our Company",
            body="Hello, please find your invoice attached.",
            attachment_path=filename
        )
        if success:
            responses.append(f"✅ Sent invoice to {invoice['customer_name']}")
        else:
            responses.append(f"❌ Failed to send invoice to {invoice['customer_name']}")
    return "\n".join(responses)

def remind_overdue_invoices_tool(_=None, **kwargs):
    """Send reminders for overdue unpaid invoices"""
    if shared_data.invoice_data is None:
        shared_data.invoice_data = get_invoice_data_from_sheet()
        
    today = datetime.today().strftime('%Y-%m-%d')
    results = []
    
    for invoice in shared_data.invoice_data:
        if invoice['status'] == 'unpaid' and invoice['due_date'] < today:
            filename = f"invoices/{invoice['customer_name'].replace(' ', '_')}_reminder.pdf"
            create_invoice_pdf(invoice, filename)

            pay_now_url = f"https://script.google.com/macros/s/AKfycbwmJ-cxnBxqtrbM0h3xobFvQ3nU9ATKhYPHflBx7fJcJTGtT2Hh3nZjwZNa28tC5b0W/exec?invoice_id={invoice['invoice_id']}"

            html_body = f"""
            <p>Dear {invoice['customer_name']},</p>
            <p>This is a reminder that your invoice dated <b>{invoice['date']}</b> is overdue.</p>
            <p>Amount Due: <b>${invoice['amount']}</b></p>
            <p>Please click the button below to mark your invoice as paid:</p>
            <a href="{pay_now_url}" style="background-color:#28a745;color:white;padding:10px 15px;text-decoration:none;border-radius:5px;">✅ Pay Now</a>
            <p>Thank you!</p>
            """

            sent = send_invoice_via_gmail(
                to_email=invoice['customer_email'],
                subject="⏰ Payment Reminder - Invoice Overdue",
                body=html_body,
                attachment_path=filename,
                is_html=True
            )

            if sent:
                results.append(f"{invoice['customer_name']} ⏰ Reminder sent.")

    return "\n".join(results) if results else "No overdue invoices."

def mark_paid_invoices_tool(_=None, **kwargs):
    """Mark paid invoices from sheet"""
    shared_data.invoice_data = get_invoice_data_from_sheet()
    updated = []
    for invoice in shared_data.invoice_data:
        if invoice['status'] == 'paid':
            updated.append(f"✅ {invoice['customer_name']}'s invoice is marked as paid.")
    return "\n".join(updated) if updated else "⚠️ No invoices marked as paid yet."

# ---------- LangChain Router Setup ----------
from langchain.agents.agent_types import AgentType

class RouterPromptTemplate(StringPromptTemplate):
    template: str
    input_variables: List[str] = ["input"]
    
    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

router_prompt = RouterPromptTemplate(
    template="""You are an expert task router. Your job is to determine whether a given task should be handled by the Payroll Agent, Invoice Agent, or Both.
    
Payroll Agent handles: payroll, salary, payslip, employee payment, attendance, deductions, bonuses
Invoice Agent handles: invoice, customer payment, billing, reminder, overdue, payment received
Both Agents handle: financial reports, monthly processing, all payments
    
Current task: {input}
    
Decision:""",
input_variables=["input"]
)

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    api_key=GROQ_API_KEY
)

router_chain = LLMChain(llm=llm, prompt=router_prompt)

def route_task(task: str) -> str:
    response = router_chain.run(input=task)
    if "payroll" in response.lower() and "invoice" in response.lower():
        return "both"
    elif "payroll" in response.lower():
        return "payroll"
    elif "invoice" in response.lower():
        return "invoice"
    else:
        return "both"  # default to both if unclear

# ---------- Agent Initialization ----------
payroll_tools = [
    Tool(name="FetchPayrollData", func=fetch_payroll_data_tool,
         description="Fetches payroll data from Google Sheets"),
    Tool(name="CalculateSalaries", func=calculate_salaries_tool,
         description="Calculates net salaries for all employees"),
    Tool(name="GeneratePayslips", func=generate_payslips_tool,
         description="Generates PDF payslips for all employees"),
    Tool(name="SendPayslips", func=send_payslips_tool,
         description="Sends generated payslips to employees")
]

invoice_tools = [
    Tool(name="CreateInvoices", func=create_all_invoice_pdfs_tool,
         description="Create invoice PDFs for all customers"),
    Tool(name="SendInvoices", func=send_all_invoices_tool,
         description="Send invoice emails to all customers"),
    Tool(name="RemindOverdueInvoices", func=remind_overdue_invoices_tool,
         description="Send reminders for overdue unpaid invoices"),
    Tool(name="MarkPaidInvoices", func=mark_paid_invoices_tool,
         description="Mark paid invoices from sheet")
]

payroll_agent = initialize_agent(
    payroll_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=7
)

invoice_agent = initialize_agent(
    invoice_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

# ---------- Main Execution ----------
def execute_task(task: str):
    agent_type = route_task(task)
    
    if agent_type == "payroll":
        print("\n🚀 Payroll Agent Processing...")
        return payroll_agent.run(task)
    elif agent_type == "invoice":
        print("\n🚀 Invoice Agent Processing...")
        return invoice_agent.run(task)
    else:
        print("\n🚀 Both Agents Processing...")
        payroll_result = payroll_agent.run(task)
        invoice_result = invoice_agent.run(task)
        return f"Payroll Results:\n{payroll_result}\n\nInvoice Results:\n{invoice_result}"

if __name__ == "__main__":
    print("Multi-Agent System Ready (LangChain Router Version)")
    while True:
        try:
            task = input("\nEnter your task (or 'quit' to exit): ")
            if task.lower() == 'quit':
                break
                
            result = execute_task(task)
            print("\nResult:\n", result)
            
        except Exception as e:
            print(f"Error: {str(e)}")

Multi-Agent System Ready (LangChain Router Version)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



🚀 Invoice Agent Processing...
Question: invoice
Thought: I need to create an invoice for a customer.
Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Here's the response:

Question: invoice
Thought: I need to create an invoice for a customer.
Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Let's continue!

Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:Let's continue!

Action: SendInvoices
Action Input: None
Observation: ✅ Sent invoice to John Doe
✅ Sent invoice to Alice Smith
Thought:Here's the response:

Question: invoice
Thought: I need to create an invoice for a customer.
Action: CreateInvoices
Action Input: None
Observation: ✅ All invoices created as PDF files.
Thought:

> Finished chain.

Result:
 Agent stopped due to iteration limit or time limit.
